Load API Keys

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

Load Contents from HLB Fixed Deposite Pages

In [2]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader([
  "https://www.hlb.com.my/en/personal-banking/fixed-deposit.html?icp=hlb-en-all-footer-txt-fd",
  "https://www.hlb.com.my/en/personal-banking/fixed-deposit/fixed-deposit-account/fixed-deposit-account.html", 
  "https://www.hlb.com.my/en/personal-banking/fixed-deposit/fixed-deposit-account/e-fixed-deposit.html",
  "https://www.hlb.com.my/en/personal-banking/fixed-deposit/fixed-deposit-account/flexi-fd.html",
  "https://www.hlb.com.my/en/personal-banking/fixed-deposit/fixed-deposit-account/senior-savers-flexi-fd.html",
  "https://www.hlb.com.my/en/personal-banking/fixed-deposit/fixed-deposit-account/junior-fixed-deposit.html",
  "https://www.hlb.com.my/en/personal-banking/fixed-deposit/fixed-deposit-account/foreign-fixed-deposit-account.html",
  "https://www.hlb.com.my/en/personal-banking/help-support/fees-and-charges/deposits.html",
])
data = loader.load()
data

[Document(page_content="\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nFixed Deposit - HLB\n\n\n\n\n\n\n\n\n\n\nFixed Deposit - HLB\n \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\npersonal\n\n\nbusiness\n\n\npriority\n\n\n\n\n\n\n\n\nIslamic\n\n\nGlobal Markets\n\n\nAssurance\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nsingapore\n\n\n\nhong kong\n\n\n\nvietnam\n\n\n\ncambodia\n\n\n\n\n\n\n                                    ENG\n\n\n\n                                                ENG\n                                            \n\n                                                BM\n                                            \n\n                                                中文\n                                            \n\n\n\n\n\n\n\n\n\nPersonal / Sole Proprietor\n\n\n\n\nBusiness\n\nHLB ConnectFirst\nHL Connect Biz\n\n\n\n\n\n\n\n\n\n\npersonal\n\n\nbusiness\n\n\npriority\n\n\n\n\n\n\n\n\nIslamic\nGlobal Markets\nAssurance\n\n\n\n\n\n\n\n\n

Split the Loaded Data into Chunks

In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=20)
texts = text_splitter.split_documents(data)
texts

[Document(page_content='Fixed Deposit - HLB\n\n\n\n\n\n\n\n\n\n\nFixed Deposit - HLB\n \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\npersonal\n\n\nbusiness\n\n\npriority\n\n\n\n\n\n\n\n\nIslamic\n\n\nGlobal Markets\n\n\nAssurance\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nsingapore\n\n\n\nhong kong\n\n\n\nvietnam\n\n\n\ncambodia\n\n\n\n\n\n\n                                    ENG\n\n\n\n                                                ENG\n                                            \n\n                                                BM\n                                            \n\n                                                中文\n                                            \n\n\n\n\n\n\n\n\n\nPersonal / Sole Proprietor\n\n\n\n\nBusiness\n\nHLB ConnectFirst\nHL Connect Biz\n\n\n\n\n\n\n\n\n\n\npersonal\n\n\nbusiness\n\n\npriority\n\n\n\n\n\n\n\n\nIslamic\nGlobal Markets\nAssurance\n\n\n\n\n\n\n\n\n\n\n\n\n                                        ABOUT US\n                                

Create Chroma DB

In [4]:
from langchain_community.vectorstores.chroma import Chroma
import chromadb
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")
chroma_client = chromadb.HttpClient(host="localhost", port=8000)
collection_name = "crc_chain"
collections = chroma_client.list_collections()
if collection_name in [collection.name for collection in collections]:
  chroma_client.delete_collection(collection_name)
vector_store = Chroma.from_documents(documents=texts, client=chroma_client, embedding=embeddings, collection_name=collection_name)
  

Create Retriever for Vector Search

In [5]:
retriever = vector_store.as_retriever(search_kwargs={"k": 15})

In [6]:
retriever.get_relevant_documents("what is fixed deposit?")

[Document(page_content='Fixed Deposit Account\n\t\t\t\t\t\t\t\n\n\n\n\n\n\n\n\n\n\n\n\nFixed Deposit\n\n\n\n\n\n\nMore Details\n\n\n\nEarn attractive interest with fixed tenures of up to 60 months\n\n\n\n\n\n\n\n\n\n\ne-Fixed Deposit\n\n\n\n\n\n\nMore Details\n\n\n\nPlace your e-Fixed Deposit online via HLB Connect\n\n\n\n\n\n\n\n\n\n\n\n\nFlexi Fixed Deposit\n\n\n\n\n\n\nMore Details\n\n\n\nEnjoy the flexibility of making early partial withdrawals\n\n\n\n\n\n\n\n\n\n\nSenior Savers Flexi Fixed Deposit\n\n\n\n\n\n\nMore Details\n\n\n\nAn attractive interest FD account for individuals aged 50 years and above\n\n\n\n\n\n\n\n\n\n\n\n\nJunior Fixed Deposit\n\n\n\n\n\n\nMore Details\n\n\n\nAn attractive interest FD account for children below the age of 18\n\n\n\n\n\n\n\n\n\n\nForeign Currency Fixed Deposit\n\n\n\n\n\n\nMore Details\n\n\n\nA convenient way to invest in foreign currency\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nCredit Card Name\n\n\n\n\n\n\n\n\n

Create LLM

In [7]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo-1106", temperature=0)

Compressor Pipeline

In [8]:
from langchain.retrievers.document_compressors import EmbeddingsFilter

embedding_filter = EmbeddingsFilter(embeddings=embeddings, similarity_threshold=0.8, k=10)

Contextual Compression

In [9]:
from langchain.retrievers import ContextualCompressionRetriever

compression_retriever = ContextualCompressionRetriever(
    base_compressor=embedding_filter, base_retriever=retriever
)

Creating Memory

In [10]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(llm=llm, memory_key="chat_history", output_key="answer", k=2, return_messages=True)

Creating Chain

In [11]:
from langchain.chains import ConversationalRetrievalChain
from langchain_community.callbacks.manager import get_openai_callback
 
chain = ConversationalRetrievalChain.from_llm(
  llm=llm,
  chain_type="stuff",
  retriever=compression_retriever,
  memory=memory,
  verbose=True,
  return_source_documents=True,
  max_tokens_limit=3000
)

def ask_question(question):
  with get_openai_callback() as cb:
    result = chain.invoke(question)
    for source_documents in result["source_documents"]:
      print(f'\n{source_documents}')
    print(f'\nDocument: {len(result["source_documents"])}')
    print(f'\nAnswer: {result["answer"]}')
    print(f'{cb}\n')

Running the Chain

In [12]:
ask_question("What is fixed deposit?")



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the user's question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
Fixed Deposit
                        



















The perfect beginning for long-term savings.









Apply Now

































I’m interested























Features & Benefits




Features & Benefits


Interest Rate (wef 21/07/2023)


Fees & Charges


Terms & Conditions













HLB Fixed Deposit account is a good start for your long-term savings plan.











Long Term

A choice of terms from 1 - 60 months.
 



                            More details

                            Less details












Interest Payout

Receive interest at maturity.
 



                            More details

                            Less details
















Fl

In [13]:
ask_question("How many types of fixed deposit does HongLeong Bank provide?")



> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:

Human: What is fixed deposit?
Assistant: A fixed deposit is a type of savings account offered by banks where you deposit a certain amount of money for a fixed period of time and in return, you receive a fixed interest rate. It's a low-risk investment option and is a good way to save money for the long term.
Follow Up Input: How many types of fixed deposit does HongLeong Bank provide?
Standalone question:

> Finished chain.


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the user's question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
Fixed Deposit | Hong Leong Bank Malaysia










Fixed Dep

In [14]:
ask_question("What is the difference between Fixed Deposit and eFixed Deposit?")



> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:

Human: What is fixed deposit?
Assistant: A fixed deposit is a type of savings account offered by banks where you deposit a certain amount of money for a fixed period of time and in return, you receive a fixed interest rate. It's a low-risk investment option and is a good way to save money for the long term.
Human: How many types of fixed deposit does HongLeong Bank provide?
Assistant: Hong Leong Bank provides various types of fixed deposit accounts, including Fixed Deposit, Junior Fixed Deposit, e-Fixed Deposit, Foreign Currency Fixed Deposit, Flexi FD Account, and Senior Savers Flexi FD. Each type of fixed deposit account offers different features and benefits to cater to the diverse needs of customers.
Follow Up Input: What is the difference between Fixed Deposit an

In [15]:
ask_question("What are the interest rates for Fixed Deposit?")



> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:

Human: How many types of fixed deposit does HongLeong Bank provide?
Assistant: Hong Leong Bank provides various types of fixed deposit accounts, including Fixed Deposit, Junior Fixed Deposit, e-Fixed Deposit, Foreign Currency Fixed Deposit, Flexi FD Account, and Senior Savers Flexi FD. Each type of fixed deposit account offers different features and benefits to cater to the diverse needs of customers.
Human: What is the difference between Fixed Deposit and eFixed Deposit?
Assistant: The main difference between a Fixed Deposit and an e-Fixed Deposit is the method of placement and management. With a Fixed Deposit, you typically visit a bank branch to place your deposit, while an e-Fixed Deposit allows you to place and manage your deposit online through the bank's digita

In [16]:
ask_question("Let's say I want to invest RM 50,000 in Fixed Deposit for 12 months. Please calculate the total amount that I can withdraw at the end of the term.")



> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:

Human: What is the difference between Fixed Deposit and eFixed Deposit?
Assistant: The main difference between a Fixed Deposit and an e-Fixed Deposit is the method of placement and management. With a Fixed Deposit, you typically visit a bank branch to place your deposit, while an e-Fixed Deposit allows you to place and manage your deposit online through the bank's digital platform. Additionally, e-Fixed Deposits often offer the flexibility of making early partial withdrawals without losing interest on the remaining balance, which may not be available with traditional Fixed Deposits.
Human: What are the interest rates for Fixed Deposit?
Assistant: The interest rates for Fixed Deposit are as follows:

- 1 month: 1.85%
- 2 months: 2.25%
- 3 months: 2.25%
- 4 months: 2.30